## 1. Preparation

**Before you start this notebook,** you need to:
- Upload the "curation.csv" file into the Files panel

Once everything is ready, you can start preparing the procedure.  
**First step:** Install all libraries  

In [ ]:
import pandas as pd
import numpy as np
import os

**Second step:** Read the dataset and show it

In [ ]:
# read the dataset
df = pd.read_csv("curation.csv")

# show data
df

## 2. Annotations with GPT

**Third step:** Make predictions with GPT API (only for the first 50 annotated sentences)  
*Note that you will need to insert here below the API key (available via the Moodle)*  
Also, you can change the prompt ("my_prompt" variable) to change the behaviour of the agent

In [ ]:
# define the API secret key
my_key = ""

# define the prompt
my_prompt = """
I will give you a csv file containing a series of book reviews with the following structure:
review_id,sentence_id,sentence
You will produce another csv file with the following structure:
review_id,sentence_id,label
The label will be assigned based on these rules:
- "evaluation" if the sentence expresses an evaluation of the reviewed book
- "other" if the sentence does not express an evaluation of the reviewed book
"""

# choose the OpenAI model
my_model = "gpt-5"

# for the annotation, we choose just the first 50 annotations
df = df[['review_id', 'sentence_id', 'sentence']]
csv_string = df[:50].to_csv(index=False)

# prepare the API client
os.environ['OPENAI_API_KEY'] = my_key
from openai import OpenAI
client = OpenAI()

# annotate with GPT
response = client.responses.create(
    model=my_model,
    instructions=my_prompt,
    input=csv_string
)

# show GPT response
print(response.output_text)

## 3. Comparison with human annotations

**Fourth step:** Convert GPT answer into a dataframe and compare with manual ones

In [ ]:
import io

# convert GPT annotations
gpt_annotations_df = pd.read_csv(io.StringIO(response.output_text))
gpt_annotations_df

# re-read the dataset
df = pd.read_csv("curation.csv")
df = df[:50]

# add GPT annotations
df['GPT'] = gpt_annotations_df['label']

display(df)

**Last step:** Calculate agreement scores (Kappa)

In [ ]:
from sklearn.metrics import cohen_kappa_score

# Calculate Cohen's Kappa scores
print("Kappa between annotators: ")
print(cohen_kappa_score(df['annotator_1'], df['annotator_2']))
print("\nKappa between annotator_1 and ChatGPT: ")
print(cohen_kappa_score(df['annotator_1'], df['GPT']))
print("\nKappa between annotator_2 and ChatGPT:")
print(cohen_kappa_score(df['annotator_2'], df['GPT']))